In [ ]:
import pandas as pd

## Subprogram

In [ ]:
def dfToDictionary(df):
  ''' Fungsi untuk membuat dictionary dari dataframe '''
  dictDf = {}
  for idx in range(len(df.index)):
    id = df.loc[idx]['id']
    # Buat Dictionary per ID
    dictDf[id] = {}
    dictDf[id]['servis'] = df.loc[idx]['servis']
    dictDf[id]['harga'] = df.loc[idx]['harga']

  return dictDf

In [ ]:
''' ==== FUZZIFICATION ==== '''

def fuzzificationMain(bengkelDict):
  ''' Fungsi untuk mengembalikan nilai fuzzy dari dataset '''
  dictFuzzyBengkel = {}
  for key in bengkelDict:
    x = bengkelDict[key]['servis']
    y = bengkelDict[key]['harga']

    # Make dictionary for fuzzy
    dictFuzzyBengkel[key] = {}
    dictFuzzyBengkel[key]['servis'] = {}
    dictFuzzyBengkel[key]['harga'] = {}

    # Fuzzy servis dan harga
    dictFuzzyBengkel = fuzzificationServis(dictFuzzyBengkel, key, x)
    dictFuzzyBengkel = fuzzificationHarga(dictFuzzyBengkel, key, y)

  return dictFuzzyBengkel

def fuzzificationServis(fuzzyDict, key, x): # Rapli
  ''' Mengembalikan nilai fuzzy dari crisp input servis '''
  # Buruk
  if x <= 50:
    fuzzyDict[key]['servis']['Buruk'] = 1
  elif 50 < x < 60:
    fuzzyDict[key]['servis']['Buruk'] = -(x - 60) / (60 - 50)
  elif x >= 60:
    fuzzyDict[key]['servis']['Buruk'] = 0

  # Lumayan
  if x <= 50 or x >= 75:
    fuzzyDict[key]['servis']['Lumayan'] = 0
  elif 50 < x < 60:
    fuzzyDict[key]['servis']['Lumayan'] = (x - 50) / (60 - 50)
  elif 60 <= x <= 70:
    fuzzyDict[key]['servis']['Lumayan'] = 1
  elif 70 < x < 75:
    fuzzyDict[key]['servis']['Lumayan'] = -(x - 75) / (75 - 70)

  # Baik
  if x <= 70 or x >= 90:
    fuzzyDict[key]['servis']['Baik'] = 0
  if 70 < x < 75:
    fuzzyDict[key]['servis']['Baik'] = (x - 70) / (75 - 70)
  if 75 <= x <= 85:
    fuzzyDict[key]['servis']['Baik'] = 1
  if 85 < x < 90:
    fuzzyDict[key]['servis']['Baik'] = -(x - 90) / (90 - 85)

  # Sangat Baik
  if x <= 85:
    fuzzyDict[key]['servis']['Sangat Baik'] = 0
  if 85 < x < 90:
    fuzzyDict[key]['servis']['Sangat Baik'] = (x - 85) / (90 - 85)
  if x >= 90:
    fuzzyDict[key]['servis']['Sangat Baik'] = 1

  return fuzzyDict

def fuzzificationHarga(fuzzyDict, key, y): # Hapian
  ''' Mengembalikan nilai fuzzy dari crisp input harga '''
  # Murah 0 - 3 - 5
  if y <= 3:
    fuzzyDict[key]['harga']['Murah'] = 1
  elif 3 < y < 5:
    fuzzyDict[key]['harga']['Murah'] = -(y - 5) / (5 - 3)
  elif y >= 5:
    fuzzyDict[key]['harga']['Murah'] = 0


  # Sedang 3 - 5 - 8
  if y <= 3 or y >= 8:
    fuzzyDict[key]['harga']['Sedang'] = 0
  elif 3 < y < 5:
    fuzzyDict[key]['harga']['Sedang'] = (y - 3) / (5 - 3)
  elif 5 <= y <= 6:
    fuzzyDict[key]['harga']['Sedang'] = 1
  elif 6 < y < 8:
    fuzzyDict[key]['harga']['Sedang'] = -(y - 8) / (8 - 6)


  # Mahal 6 - 8 - 10
  if y <= 6:
    fuzzyDict[key]['harga']['Mahal'] = 0
  if 6 < y < 8:
    fuzzyDict[key]['harga']['Mahal'] = (y - 6) / (8 - 6)
  if y >= 8:
    fuzzyDict[key]['harga']['Mahal'] = 1

  return fuzzyDict

# Example Hasil:
#  {1: {'harga': {'Mahal': 0.5, 'Murah': 0, 'Sedang': 0.5},
#    'servis': {'Baik': 0, 'Buruk': 0.2, 'Lumayan': 0.8, 'Sangat Baik': 0}},
#  2: {'harga': {'Mahal': 0, 'Murah': 1, 'Sedang': 0},
#    'servis': {'Baik': 0, 'Buruk': 0.6, 'Lumayan': 0.4, 'Sangat Baik': 0}},

In [ ]:
''' ==== INFERENCE ==== '''

def inferenceMain(fuzziedDict):
  ''' Mengembalikan hasil inferensi dari nilai fuzzy harga dan servis yang dipunya '''
  # Buat Dictionary untuk hasil inferensi
  dictInferenced = {}
  for id in fuzziedDict:
    dictInferenced[id] = {}

    # List untuk nampung value hasil inference per bengkel
    listBuruk = []
    listBiasaAja = []
    listRecommended = []

    for varHarga, valueHarga in fuzziedDict[id]['harga'].items():
      for varServis, valueServis in fuzziedDict[id]['servis'].items():
        minAnd = min(valueHarga, valueServis)
        score = inferenceTable(varServis, varHarga)

        # Append ke List untuk nanti di max() / or
        if score == 'Buruk':
          listBuruk.append(minAnd)
        elif score == 'B aja':
          listBiasaAja.append(minAnd)
        elif score == 'Recommended':
          listRecommended.append(minAnd)

    dictInferenced[id]['Buruk'] = max(listBuruk)
    dictInferenced[id]['B aja'] = max(listBiasaAja)
    dictInferenced[id]['Recommended'] = max(listRecommended)

  return dictInferenced

# Example Dictionary Inference yang Udah Jadi
# {1 : {'Buruk' : 0, 'B aja' : 0, 'Recommended' : 0},
# 2 : {'Buruk' : 0, 'B aja' : 0, 'Recommended' : 0}}

def inferenceTable(keyServis, keyHarga):
  if keyServis == 'Buruk':
    if keyHarga == 'Murah':
      hasil = 'Buruk'
    elif keyHarga == 'Sedang':
      hasil = 'Buruk'
    elif keyHarga == 'Mahal':
      hasil = 'Buruk'

  elif keyServis == 'Lumayan':
    if keyHarga == 'Murah':
      hasil = 'B aja'
    elif keyHarga == 'Sedang':
      hasil = 'B aja'
    elif keyHarga == 'Mahal':
      hasil = 'Buruk'

  elif keyServis == 'Baik':
    if keyHarga == 'Murah':
      hasil = 'Recommended'
    elif keyHarga == 'Sedang':
      hasil = 'Recommended'
    elif keyHarga == 'Mahal':
      hasil = 'B aja'

  elif keyServis == 'Sangat Baik':
    if keyHarga == 'Murah':
      hasil = 'Recommended'
    elif keyHarga == 'Sedang':
      hasil = 'Recommended'
    elif keyHarga == 'Mahal':
      hasil = 'Recommended'

  return hasil

In [ ]:
def defuzziSugeno(inferencedDataset):
  ''' Mengembalikan nilai hasil deffuzification menggunakan metode sugeno '''
  dictdefuzied = {}
  for id in inferencedDataset:
    hasil = (inferencedDataset[id]['B aja']*70) + (inferencedDataset[id]['Buruk']*50) + (inferencedDataset[id]['Recommended']*100)
    pembagi = inferencedDataset[id]['B aja'] + inferencedDataset[id]['Buruk'] + inferencedDataset[id]['Recommended']
    defuzzi = hasil/pembagi
    dictdefuzied[id] = defuzzi

  return dictdefuzied

# Example Output
# id bengkel : hasil defuzzi
# {1 : 50,
#  2 : 10,
#  3 : 90}

In [ ]:
def makeDataframeTopTen(sortedDataset):
  ''' Fungsi untuk mengembalikan dataframe dari dataset yang sudah di sort '''
  listID = []
  listScore = []
  for i in range(len(sortedDataset)):
    listID.append(sortedDataset[i][0])
    listScore.append(sortedDataset[i][1])

  final = zip(listID, listScore)
  dfFinal = pd.DataFrame(final, columns = ["ID", "Hasil"])

  return dfFinal

## Preparation

In [ ]:
# Download Dataset
!gdown --id "1XGEMtjvPF2PUV0Mwklnpk0Ev9AwJitFq"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1XGEMtjvPF2PUV0Mwklnpk0Ev9AwJitFq
To: /content/bengkel.xlsx
100% 16.9k/16.9k [00:00<00:00, 22.3MB/s]


In [ ]:
# Membaca file excel
bengkelDf = pd.read_excel('bengkel.xlsx')
bengkelDf

,id,servis,harga
0,1,58,7
1,2,54,1
2,3,98,2
3,4,52,4
4,5,11,4
...,...,...,...
95,96,30,1
96,97,25,3
97,98,27,10
98,99,8,6


In [ ]:
# Mengubah dataframe menjadi dictionary
dictBengkel = dfToDictionary(bengkelDf)
dictBengkel

{1: {'harga': 7, 'servis': 58},
 2: {'harga': 1, 'servis': 54},
 3: {'harga': 2, 'servis': 98},
 4: {'harga': 4, 'servis': 52},
 5: {'harga': 4, 'servis': 11},
 6: {'harga': 10, 'servis': 59},
 7: {'harga': 8, 'servis': 61},
 8: {'harga': 10, 'servis': 30},
 9: {'harga': 1, 'servis': 45},
 10: {'harga': 9, 'servis': 36},
 11: {'harga': 5, 'servis': 10},
 12: {'harga': 7, 'servis': 38},
 13: {'harga': 3, 'servis': 80},
 14: {'harga': 8, 'servis': 31},
 15: {'harga': 5, 'servis': 78},
 16: {'harga': 6, 'servis': 82},
 17: {'harga': 3, 'servis': 70},
 18: {'harga': 9, 'servis': 3},
 19: {'harga': 3, 'servis': 42},
 20: {'harga': 10, 'servis': 49},
 21: {'harga': 2, 'servis': 48},
 22: {'harga': 9, 'servis': 79},
 23: {'harga': 4, 'servis': 18},
 24: {'harga': 9, 'servis': 100},
 25: {'harga': 10, 'servis': 61},
 26: {'harga': 2, 'servis': 4},
 27: {'harga': 8, 'servis': 59},
 28: {'harga': 3, 'servis': 44},
 29: {'harga': 8, 'servis': 11},
 30: {'harga': 6, 'servis': 7},
 31: {'harga': 9,

## Main

In [ ]:
# Fuzziffication Dataset
fuzziedDataset = fuzzificationMain(dictBengkel)
fuzziedDataset

{1: {'harga': {'Mahal': 0.5, 'Murah': 0, 'Sedang': 0.5},
  'servis': {'Baik': 0, 'Buruk': 0.2, 'Lumayan': 0.8, 'Sangat Baik': 0}},
 2: {'harga': {'Mahal': 0, 'Murah': 1, 'Sedang': 0},
  'servis': {'Baik': 0, 'Buruk': 0.6, 'Lumayan': 0.4, 'Sangat Baik': 0}},
 3: {'harga': {'Mahal': 0, 'Murah': 1, 'Sedang': 0},
  'servis': {'Baik': 0, 'Buruk': 0, 'Lumayan': 0, 'Sangat Baik': 1}},
 4: {'harga': {'Mahal': 0, 'Murah': 0.5, 'Sedang': 0.5},
  'servis': {'Baik': 0, 'Buruk': 0.8, 'Lumayan': 0.2, 'Sangat Baik': 0}},
 5: {'harga': {'Mahal': 0, 'Murah': 0.5, 'Sedang': 0.5},
  'servis': {'Baik': 0, 'Buruk': 1, 'Lumayan': 0, 'Sangat Baik': 0}},
 6: {'harga': {'Mahal': 1, 'Murah': 0, 'Sedang': 0},
  'servis': {'Baik': 0, 'Buruk': 0.1, 'Lumayan': 0.9, 'Sangat Baik': 0}},
 7: {'harga': {'Mahal': 1, 'Murah': 0, 'Sedang': 0},
  'servis': {'Baik': 0, 'Buruk': 0, 'Lumayan': 1, 'Sangat Baik': 0}},
 8: {'harga': {'Mahal': 1, 'Murah': 0, 'Sedang': 0},
  'servis': {'Baik': 0, 'Buruk': 1, 'Lumayan': 0, 'Sangat 

In [ ]:
# Inference Dataset
inferencedDataset = inferenceMain(fuzziedDataset)
inferencedDataset

{1: {'B aja': 0.5, 'Buruk': 0.5, 'Recommended': 0},
 2: {'B aja': 0.4, 'Buruk': 0.6, 'Recommended': 0},
 3: {'B aja': 0, 'Buruk': 0, 'Recommended': 1},
 4: {'B aja': 0.2, 'Buruk': 0.5, 'Recommended': 0},
 5: {'B aja': 0, 'Buruk': 0.5, 'Recommended': 0},
 6: {'B aja': 0, 'Buruk': 0.9, 'Recommended': 0},
 7: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 8: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 9: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 10: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 11: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 12: {'B aja': 0, 'Buruk': 0.5, 'Recommended': 0},
 13: {'B aja': 0, 'Buruk': 0, 'Recommended': 1},
 14: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 15: {'B aja': 0, 'Buruk': 0, 'Recommended': 1},
 16: {'B aja': 0, 'Buruk': 0, 'Recommended': 1},
 17: {'B aja': 1, 'Buruk': 0, 'Recommended': 0},
 18: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 19: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 20: {'B aja': 0, 'Buruk': 1, 'Recommended': 0},
 21: {'B aj

In [ ]:
# Defuzzification Dataset
defuziedDataset = defuzziSugeno(inferencedDataset)
defuziedDataset

{1: 60.0,
 2: 58.0,
 3: 100.0,
 4: 55.714285714285715,
 5: 50.0,
 6: 50.0,
 7: 50.0,
 8: 50.0,
 9: 50.0,
 10: 50.0,
 11: 50.0,
 12: 50.0,
 13: 100.0,
 14: 50.0,
 15: 100.0,
 16: 100.0,
 17: 70.0,
 18: 50.0,
 19: 50.0,
 20: 50.0,
 21: 50.0,
 22: 70.0,
 23: 50.0,
 24: 100.0,
 25: 50.0,
 26: 50.0,
 27: 50.0,
 28: 50.0,
 29: 50.0,
 30: 50.0,
 31: 66.0,
 32: 50.0,
 33: 50.0,
 34: 100.0,
 35: 50.0,
 36: 50.0,
 37: 50.0,
 38: 50.0,
 39: 55.714285714285715,
 40: 50.0,
 41: 50.0,
 42: 100.0,
 43: 50.0,
 44: 70.0,
 45: 50.0,
 46: 50.0,
 47: 50.0,
 48: 70.0,
 49: 50.0,
 50: 50.0,
 51: 50.0,
 52: 100.0,
 53: 50.0,
 54: 50.0,
 55: 50.0,
 56: 50.0,
 57: 50.0,
 58: 50.0,
 59: 50.0,
 60: 100.0,
 61: 50.0,
 62: 50.0,
 63: 85.0,
 64: 50.0,
 65: 50.0,
 66: 50.0,
 67: 50.0,
 68: 68.0,
 69: 76.0,
 70: 70.0,
 71: 50.0,
 72: 50.0,
 73: 50.0,
 74: 58.888888888888886,
 75: 70.0,
 76: 50.0,
 77: 50.0,
 78: 50.0,
 79: 82.0,
 80: 50.0,
 81: 50.0,
 82: 50.0,
 83: 50.0,
 84: 50.0,
 85: 50.0,
 86: 50.0,
 87: 61.1111

In [ ]:
# Menyortir hasil defuzzification menjadi descending berdasarkan value
sortedDefuziedDataset = sorted(defuziedDataset.items(), key=lambda item: item[1], reverse=True)
sortedDefuziedDataset

[(3, 100.0),
 (13, 100.0),
 (15, 100.0),
 (16, 100.0),
 (24, 100.0),
 (34, 100.0),
 (42, 100.0),
 (52, 100.0),
 (60, 100.0),
 (91, 100.0),
 (92, 100.0),
 (63, 85.0),
 (79, 82.0),
 (69, 76.0),
 (17, 70.0),
 (22, 70.0),
 (44, 70.0),
 (48, 70.0),
 (70, 70.0),
 (75, 70.0),
 (68, 68.0),
 (31, 66.0),
 (87, 61.11111111111111),
 (1, 60.0),
 (74, 58.888888888888886),
 (2, 58.0),
 (4, 55.714285714285715),
 (39, 55.714285714285715),
 (5, 50.0),
 (6, 50.0),
 (7, 50.0),
 (8, 50.0),
 (9, 50.0),
 (10, 50.0),
 (11, 50.0),
 (12, 50.0),
 (14, 50.0),
 (18, 50.0),
 (19, 50.0),
 (20, 50.0),
 (21, 50.0),
 (23, 50.0),
 (25, 50.0),
 (26, 50.0),
 (27, 50.0),
 (28, 50.0),
 (29, 50.0),
 (30, 50.0),
 (32, 50.0),
 (33, 50.0),
 (35, 50.0),
 (36, 50.0),
 (37, 50.0),
 (38, 50.0),
 (40, 50.0),
 (41, 50.0),
 (43, 50.0),
 (45, 50.0),
 (46, 50.0),
 (47, 50.0),
 (49, 50.0),
 (50, 50.0),
 (51, 50.0),
 (53, 50.0),
 (54, 50.0),
 (55, 50.0),
 (56, 50.0),
 (57, 50.0),
 (58, 50.0),
 (59, 50.0),
 (61, 50.0),
 (62, 50.0),
 (64, 5

In [ ]:
# Mengubah hasil sorted defuzzi menjadi dataframe agar bisa diubah ke file excel kembali
dfHasilAkhir = makeDataframeTopTen(sortedDefuziedDataset)
dfHasilAkhir

,ID,Hasil
0,3,100.0
1,13,100.0
2,15,100.0
3,16,100.0
4,24,100.0
...,...,...
95,96,50.0
96,97,50.0
97,98,50.0
98,99,50.0


In [ ]:
# Membuat file excel dari dataframe top 10 (row 0 - 9)
dfHasilAkhir.head(10).to_excel("peringkat.xlsx")